In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math
import yfinance as yf

ticker1="^IXIC"
ticker2="^NSEI"

df1=yf.download(ticker1, start="2010-1-1", end="2023-5-1")
df2=yf.download(ticker2, start="2010-1-1", end="2023-5-1")

nasdaq_data = df1
nse_data = df2

nasdaq_close = nasdaq_data['Close']
nse_close = nse_data['Close']

# Correlation coefficient
correlation_coefficient = nasdaq_close.corr(nse_close)

print("Correlation coefficient: ", correlation_coefficient)

# Correlation coefficient is 0.9513138758576783 which is close to 1. Thus it depicts positive correlation between the indices

#lag analysis
# max_lag = 30
# correlation_results = []
# for lag in range(-max_lag, max_lag+1):
#     shifted_nasdaq_close = nasdaq_close.shift(lag)
#     correlation_coefficient = shifted_nasdaq_close.corr(nse_close)
#     correlation_results.append((lag, correlation_coefficient))

# correlation_results.sort(key=lambda x: abs(x[1]), reverse=True)

# print("Top Lead-Lag Relationships:")
# for lag, correlation_coefficient in correlation_results[:5]:
#     print(f"Lag: {lag}, Correlation Coefficient: {correlation_coefficient}")

#Positive results shows that NASDAQ index leads NSE index

#Keltner Channel, Bollinger Bands, and MACD indicators for NASDAQ
def calculate_average_true_range(data, window=14):
    high = data['High']
    low = data['Low']
    close = data['Close']
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(window).mean()
    return atr

def calculate_keltner_channel(data, window=20, multiplier=2):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    atr = calculate_average_true_range(data, window)
    upper_band = typical_price.rolling(window).mean() + multiplier * atr
    lower_band = typical_price.rolling(window).mean() - multiplier * atr
    middle_band = typical_price.rolling(window).mean()
    return upper_band, middle_band, lower_band

def calculate_bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['Close'].rolling(window).mean()
    rolling_std = data['Close'].rolling(window).std()
    upper_band = rolling_mean + num_std * rolling_std
    lower_band = rolling_mean - num_std * rolling_std
    return upper_band, lower_band

def calculate_macd(data, slow_period=26, fast_period=12, signal_period=9):
    exp1 = data['Close'].ewm(span=fast_period, adjust=False).mean()
    exp2 = data['Close'].ewm(span=slow_period, adjust=False).mean()
    macd_line = exp1 - exp2
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

# Calculate the Keltner Channel
keltner_upper, keltner_middle, keltner_lower = calculate_keltner_channel(df1)

# Calculate the Bollinger Bands
bollinger_upper, bollinger_lower = calculate_bollinger_bands(df1)

# Calculate the MACD
macd_line, signal_line, histogram = calculate_macd(df1)

# Plotting the indicators
plt.figure(figsize=(12, 6))
plt.plot(df1['Close'], label='Close')
plt.plot(keltner_upper, label='Keltner Upper', linestyle='--')
plt.plot(keltner_middle, label='Keltner Middle')
plt.plot(keltner_lower, label='Keltner Lower', linestyle='--')
plt.plot(bollinger_upper, label='Bollinger Upper', linestyle='--')
plt.plot(bollinger_lower, label='Bollinger Lower', linestyle='--')
plt.legend()

plt.figure(figsize=(12, 4))
plt.plot(macd_line, label='MACD Line')
plt.plot(signal_line, label='Signal Line')
plt.bar(df1.index, histogram, label='MACD Histogram')
plt.legend()
plt.show()



